import all the relevant libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stats
import io
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split


upload the db file

In [ ]:

from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

#**helper functions**

In [ ]:
def calcCrossValidation_RMSE(model, df, predictors, outcome):


  # Perform k-fold cross-validation with 10 folds
  kf = KFold(n_splits=df.shape[0])
  accuracy = []
  total_RMSE=0
  for train, test in kf.split(df):
      # print(train)
      # Filter training data
      train_predictors = (df[predictors].iloc[train, :])
      # The target we're using to train the algorithm.
      train_target = df[outcome].iloc[train]
      test_predictors = (df[predictors].iloc[test, :])
      # print(train_predictors)

      # The target we're using to train the algorithm.
      test_target = df[outcome].iloc[test]

      # Training the algorithm using the predictors and target.
      # print(train_target)
      model.fit(train_predictors, train_target)
      predictions = model.predict(test_predictors)
      # print(predictions)
      rmse = sqrt(mean_squared_error(test_target, predictions))  
      # print(rmse)
      total_RMSE += rmse

  return total_RMSE/df.shape[0]
  # return 0

In [ ]:
def calcCrossValidation_Matrix(model, df, predictors, outcome):

  total_tp = 0
  total_tn = 0
  total_fp = 0
  total_fn = 0

  # Perform k-fold cross-validation with 10 folds
  kf = KFold(n_splits=df.shape[0])
  accuracy = []
  for train, test in kf.split(df):
      # print("1")

      # Filter training data
      train_predictors = (df[predictors].iloc[train, :])
      # The target we're using to train the algorithm.
      train_target = df[outcome].iloc[train]
      # print("1")

      test_predictors = (df[predictors].iloc[test, :])
      # The target we're using to train the algorithm.
      test_target = df[outcome].iloc[test]
      # print("1")


      # Training the algorithm using the predictors and target.
      model.fit(train_predictors, train_target)
      predictions = model.predict(test_predictors)
      # cm = confusion_matrix(test_target, predictions)
      # print(cm)
      # print("1")
      # print("test_target")
      test_target = test_target.to_numpy()
      # print(test_target)
      # np.array(df[outcome].values.tolist())
      # print(type(test_target))
      # print("predictions")
      # print(predictions)
      # print(type(predictions))

      tn, fp, fn, tp = perf_measure(test_target, predictions)
      # print("1")

      total_tp += tp
      total_tn += tn
      total_fp += fp
      total_fn += fn

  # print(total_tn, total_fp, total_fn, total_tp)
  return total_tn, total_fp, total_fn, total_tp 

In [ ]:
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i]==y_hat[i]==1:
           TP += 1
        if y_hat[i]==1 and y_actual[i]!=y_hat[i]:
           FP += 1
        if y_actual[i]==y_hat[i]==0:
           TN += 1
        if y_hat[i]==0 and y_actual[i]!=y_hat[i]:
           FN += 1
    # print(TN, FP, FN, TP)
    return(TN, FP, FN, TP)

In [ ]:
def calc_specificity(tn, fp, fn, tp):
  return tn / (tn +fp)

In [ ]:
def calc_sensitivity(tn, fp, fn, tp):
  return tp / (tp +fn)

new function Shira

In [ ]:
df_experiments = pd.DataFrame(columns=['model','missing values','specificity', 'sensitivity', 'RMSE']) 

In [ ]:
def update_experiments_first(df, outcome_var, model_type, all_features, missing_values):
  for value in missing_values:
    all_features.remove(value)
  predictor_var = all_features
  if (model_type=='RandomForestClassifier'):
    model = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
  # elif (model_type=='LogisticRegression'):
  add_sensitivity_specifity(model, df, predictor_var, outcome_var, model_type, missing_values)

In [ ]:
def add_sensitivity_specifity(model, df, predictor_var, outcome_var, model_type, missing_values):
  tn, fp, fn, tp = calcCrossValidation_Matrix(model, df, predictor_var, outcome_var)
  specificity = calc_specificity(tn, fp, fn, tp)
  sensitivity = calc_sensitivity(tn, fp, fn, tp)
  specifity_sensitivity_list = [specificity, sensitivity]
  update_results(specifity_sensitivity_list, missing_values, model_type)

In [ ]:
def update_results(results_list, missing_values, model_type):
  # df_experiments = pd.DataFrame(columns=['model','missing values','specificity', 'sensitivity', 'RMSE']) 
  # df_experiments = pd.DataFrame(['model','missing values','specificity', 'sensitivity', 'RMSE']) 
  if (len(results_list)==2):
    new_record = {'model': model_type,'missing values': missing_values,'specificity': results_list[0], 'sensitivity':results_list[1]}
  else:
    new_record = {'model': model_type,'missing values': missing_values,'specificity': results_list[0], 'sensitivity':results_list[1], 'RMSE':results_list[2]}
  df_experiments = df_experiments.append(new_record, ignore_index=True)
  print(df_experiments)
  # return df_experiments



#**all untreated**

In [ ]:
const_seed = 42

In [ ]:
df_untreated = pd.read_pickle("/content/gdrive/My Drive/Colab Notebooks/MS_Notebooks/DF_only_untreated.pkl")

In [ ]:
df_untreated

In [ ]:
def min_max_normalize_features(dataframe, numeric_list):
    for num_feature in numeric_list:
        dataframe[num_feature] = (dataframe[num_feature] - dataframe[num_feature].min()) / (
                dataframe[num_feature].max() - dataframe[num_feature].min())
    return dataframe

In [ ]:
numeric_features = ['TIM3_RQ', 'TIGIT_RQ','LAG3_RQ','age_at_diagnosis','MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells']
df_untreated = min_max_normalize_features(df_untreated, numeric_features)
df_untreated

In [ ]:
categorical_features = ['gender','oligoclonal_band','diagnosis']
df_untreated = pd.get_dummies(df_untreated, columns=categorical_features, drop_first=True)
df_untreated.drop(['treatment_when_sample_was_taken', 'sample_code'], axis='columns', inplace=True)



In [ ]:
df_untreated.columns.values

array(['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ', 'age_at_diagnosis',
       'primary_EDSS_at_diagnosis', '1_years_EDSS_from_sample_date',
       '5_years_from_sample', 'EDSS_post_10_years', 'MRI_lesion_mass',
       'Lymph_Cells', 'PMN_Cells', 'Cells', 'gender_M',
       'oligoclonal_band_POS', 'diagnosis_SPMS'], dtype=object)

all features

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


coefficients = pd.concat([pd.DataFrame(predictor_var1), pd.DataFrame(np.transpose(logisticRegrDF.coef_))], axis=1)
coefficients.columns = ['feature_name', 'coefficient_val']
coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_no_abs.xlsx')
coefficients['coefficient_val'] = coefficients['coefficient_val'].abs()

coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_abs.xlsx')

print(coefficients)


0.8714285714285714
0.47058823529411764
                 feature_name  coefficient_val
0                     TIM3_RQ         1.381569
4   primary_EDSS_at_diagnosis         0.884037
9                    gender_M         0.725555
5             MRI_lesion_mass         0.660757
10       oligoclonal_band_POS         0.454942
7                   PMN_Cells         0.431771
3            age_at_diagnosis         0.365317
6                 Lymph_Cells         0.339908
2                     LAG3_RQ         0.259506
8                       Cells         0.045944
1                    TIGIT_RQ         0.043782


In [ ]:
rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
importances = rf.feature_importances_
importances

0.9142857142857143
0.38235294117647056


array([0.11830236, 0.09396613, 0.09497382, 0.18161032, 0.24306953,
       0.13928189, 0.00983947, 0.0321862 , 0.04475534, 0.02766252,
       0.01435243])

** new ** NO - TIGIT, LAg3, PD-1, Cells in CSF 

In [ ]:
rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var = ['TIM3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'


tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
importances = rf.feature_importances_
importances



0.8571428571428571
0.5588235294117647


array([0.26392673, 0.2136695 , 0.30500617, 0.17771591, 0.01704241,
       0.02263928])

In [ ]:

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


coefficients = pd.concat([pd.DataFrame(predictor_var), pd.DataFrame(np.transpose(logisticRegrDF.coef_))], axis=1)
coefficients.columns = ['feature_name', 'coefficient_val']
coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_no_abs.xlsx')
coefficients['coefficient_val'] = coefficients['coefficient_val'].abs()

coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_abs.xlsx')

print(coefficients)

0.8857142857142857
0.47058823529411764
                feature_name  coefficient_val
0                    TIM3_RQ         1.394222
2  primary_EDSS_at_diagnosis         0.881592
4                   gender_M         0.728825
3            MRI_lesion_mass         0.656682
5       oligoclonal_band_POS         0.444508
1           age_at_diagnosis         0.362760


** new ** top 7 combination

In [ ]:
predictor_var = ['PMN_Cells', 'TIM3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'

In [ ]:
rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)

tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
importances = rf.feature_importances_
importances



0.8714285714285714
0.5294117647058824


array([0.02621432, 0.24444123, 0.20383393, 0.28826469, 0.1759557 ,
       0.03472829, 0.02656184])

In [ ]:

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


coefficients = pd.concat([pd.DataFrame(predictor_var), pd.DataFrame(np.transpose(logisticRegrDF.coef_))], axis=1)
coefficients.columns = ['feature_name', 'coefficient_val']
coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_no_abs.xlsx')
coefficients['coefficient_val'] = coefficients['coefficient_val'].abs()

coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_abs.xlsx')

print(coefficients)


0.8857142857142857
0.4411764705882353
                feature_name  coefficient_val
1                    TIM3_RQ         1.406916
3  primary_EDSS_at_diagnosis         0.877842
5                   gender_M         0.731453
4            MRI_lesion_mass         0.664858
0                  PMN_Cells         0.440652
6       oligoclonal_band_POS         0.437463
2           age_at_diagnosis         0.365001


no genes

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


coefficients = pd.concat([pd.DataFrame(predictor_var1), pd.DataFrame(np.transpose(logisticRegrDF.coef_))], axis=1)
coefficients.columns = ['feature_name', 'coefficient_val']
coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_no_abs.xlsx')
coefficients['coefficient_val'] = coefficients['coefficient_val'].abs()

coefficients = coefficients.sort_values(by=['coefficient_val'], ascending=False)
coefficients.to_excel('coef_abs.xlsx')

print(coefficients)

0.8714285714285714
0.47058823529411764
                feature_name  coefficient_val
1  primary_EDSS_at_diagnosis         0.876481
6                   gender_M         0.753126
2            MRI_lesion_mass         0.669394
7       oligoclonal_band_POS         0.435083
4                  PMN_Cells         0.398075
3                Lymph_Cells         0.352764
0           age_at_diagnosis         0.351972
5                      Cells         0.054754


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8285714285714286
0.47058823529411764


no TIM3

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


0.8857142857142857
0.4411764705882353


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.9142857142857143
0.3235294117647059


no TIGIT_RQ

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.9
0.38235294117647056


no LAG3

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8571428571428571
0.4411764705882353


no TIM3 and LAG3

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


0.8714285714285714
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIGIT_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8857142857142857
0.4117647058823529


no TIM3 and TIGIT

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


0.8857142857142857
0.4411764705882353


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8428571428571429
0.4411764705882353


no LAG3 and TIGIT

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
randomForestRegr = RandomForestRegressor(n_estimators = 10, random_state=const_seed)
predictor_rmse = ['TIM3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_rmse = 'EDSS_post_10_years'
rmse = calcCrossValidation_RMSE(randomForestRegr, df_untreated, predictor_rmse, outcome_rmse)
print(rmse)

0.8714285714285714
0.47058823529411764
1.7920673076923062


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.47058823529411764


no MRI

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))


0.8714285714285714
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.9
0.4117647058823529


no OCB

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8857142857142857
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8857142857142857
0.47058823529411764


no total cells

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.9
0.4411764705882353


no lymph cells

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Cells','PMN_Cells','gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
randomForestRegr = RandomForestRegressor(n_estimators = 10, random_state=const_seed)
predictor_rmse = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Cells','PMN_Cells','gender_M', 'oligoclonal_band_POS']
outcome_rmse = 'EDSS_post_10_years'
rmse = calcCrossValidation_RMSE(randomForestRegr, df_untreated, predictor_rmse, outcome_rmse)
print(rmse)

0.8714285714285714
0.4411764705882353
1.7764423076923075


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Cells','PMN_Cells','gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8857142857142857
0.38235294117647056


no PMM cells

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Cells','Lymph_Cells','gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8571428571428571
0.4411764705882353


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Cells','Lymph_Cells','gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.38235294117647056


no MRI, cells

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'PMN_Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
randomForestRegr = RandomForestRegressor(n_estimators = 10,random_state=const_seed)
predictor_rmse = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'PMN_Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_rmse = 'EDSS_post_10_years'
rmse = calcCrossValidation_RMSE(randomForestRegr, df_untreated, predictor_rmse, outcome_rmse)
print(rmse)

0.8571428571428571
0.47058823529411764
1.7528846153846147


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'PMN_Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.9
0.38235294117647056


no MRI, PMM cells

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
randomForestRegr = RandomForestRegressor(n_estimators = 10,random_state=const_seed)
predictor_rmse = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_rmse = 'EDSS_post_10_years'
rmse = calcCrossValidation_RMSE(randomForestRegr, df_untreated, predictor_rmse, outcome_rmse)
print(rmse)

0.8571428571428571
0.47058823529411764
1.7870192307692305


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.4117647058823529


no TIGIT, cells

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'MRI_lesion_mass', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'PMN_Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'MRI_lesion_mass', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'PMN_Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8571428571428571
0.4411764705882353


NO TIGIT, PMM cells

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'MRI_lesion_mass', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'MRI_lesion_mass', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Cells', 'Lymph_Cells', 'gender_M', 'oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8285714285714286
0.38235294117647056


no MRI, LAG3

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8571428571428571
0.47058823529411764


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'TIGIT_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8571428571428571
0.5294117647058824


no MRI, TIM3, LAG3

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIGIT_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
randomForestRegr = RandomForestRegressor(n_estimators = 10,random_state=const_seed)
predictor_rmse = ['TIGIT_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_rmse = 'EDSS_post_10_years'
rmse = calcCrossValidation_RMSE(randomForestRegr, df_untreated, predictor_rmse, outcome_rmse)
print(rmse)

0.8857142857142857
0.47058823529411764
1.9478365384615384


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIGIT_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8285714285714286
0.35294117647058826


no MRI, TIM3, TIGIT

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
randomForestRegr = RandomForestRegressor(n_estimators = 10,random_state=const_seed)
predictor_rmse = ['LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_rmse = 'EDSS_post_10_years'
rmse = calcCrossValidation_RMSE(randomForestRegr, df_untreated, predictor_rmse, outcome_rmse)
print(rmse)

0.9
0.47058823529411764
1.6911057692307692


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8428571428571429
0.4117647058823529


no MRI, TIGIT, LAG3

In [ ]:
from sklearn.metrics import confusion_matrix

logisticRegrDF = LogisticRegression(max_iter = 4000,random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(logisticRegrDF, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))
randomForestRegr = RandomForestRegressor(n_estimators = 10,random_state=const_seed)
predictor_rmse = ['TIM3_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_rmse = 'EDSS_post_10_years'
rmse = calcCrossValidation_RMSE(randomForestRegr, df_untreated, predictor_rmse, outcome_rmse)
print(rmse)

0.8571428571428571
0.47058823529411764
1.6663461538461541


In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['TIM3_RQ', 'age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8714285714285714
0.5


In [ ]:
df_untreated.shape

(104, 15)

# shira new

In [ ]:
from sklearn.metrics import confusion_matrix

# rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
all_features = ['TIM3_RQ', 'TIGIT_RQ', 'LAG3_RQ','age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
missing_val = ['TIM3_RQ','TIGIT_RQ', 'LAG3_RQ']
outcome_var = 'diagnosis_SPMS'
model_type = 'RandomForestClassifier'
update_experiments_first(df_untreated, outcome_var, model_type, all_features, missing_val)
df_experiments
# tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
# print(calc_specificity(tn, fp, fn, tp))
# print(calc_sensitivity(tn, fp, fn, tp))

UnboundLocalError: ignored

In [ ]:
from sklearn.metrics import confusion_matrix

rf = RandomForestClassifier(n_estimators=25, min_samples_split=9, max_depth=15, max_features='auto',random_state=const_seed)
predictor_var1 = ['age_at_diagnosis', 'primary_EDSS_at_diagnosis', 'MRI_lesion_mass','Lymph_Cells','PMN_Cells','Cells','gender_M','oligoclonal_band_POS']
outcome_var = 'diagnosis_SPMS'
tn, fp, fn, tp = calcCrossValidation_Matrix(rf, df_untreated, predictor_var1, outcome_var)
print(calc_specificity(tn, fp, fn, tp))
print(calc_sensitivity(tn, fp, fn, tp))

0.8285714285714286
0.47058823529411764
